# 0. Installation and import the libraries

FIRST THING FIRST. 

Install sylfenUtils==1.5 in a virtual environment. Replace sylfen-user by your own account on Gaston in the following command and enter your password when asked:
```
pip install sylfenUtils @git+ssh://<sylfen-user>@cheylas.sylfen.com/home/<sylfen-user>/gitRepos/sylfenUtils.git@1.5
```
In order to run your jupyter-notebook with your virtual environment follow instructions [here](https://queirozf.com/entries/jupyter-kernels-how-to-add-change-remove).
Then import the libraries.

In [ ]:
from sylfenUtils import comUtils
import os,sys,re, pandas as pd

In [ ]:
pip list

# 1.Build the configuration of your project
For convenience put your configuration work in a conf.py file which should serve as an object with all
the important information. 
It will be called as a module(import conf) 
You can also create a class and instanciate it (conf=Conf())

In [ ]:
class Conf():### not necessary if you call the module of course.
    pass

conf=Conf()

Let's say you have a modbus table of a device in a "modbus_dummy.csv"(download it [here](wiki.sylfen.com/notebooks/data/modbus_dummy.csv) ) file and you can call this device "dummy" 

In [ ]:
conf.dummy_modbus_map=pd.read_csv('modbus_dummy.csv',index_col=0)
conf.dummy_modbus_map

To use the dumper class of the comUtils module of the sylfenUtils library you need to build a df_plc dataframe with at least following columns:
- TAGS(as index)
- DESCRIPTION
- UNITE
- DATATYPE. 
For example you can build it from your dummy_modbus_device as follow:

In [ ]:
df_plc=conf.dummy_modbus_map[['description','unit','type']]
df_plc.columns=['DESCRIPTION','UNITE','DATATYPE']
conf.dummy_df_plc=df_plc

Your configuration is done now. 
A trick in order not to have to generate the whole configuration you can store the result in a .pkl file with the pandas function **to_pickle** and put all the steps need to recreate your whole configuration in a function ("createMyConf" for example). Then you can call this function to regenerate your configuration. Otherwise just load the .pkl file(will be much faster if you have a lot of steps).

Now you need to precise the parameters of the database you want to use.
For the data acquisition you NEED TO INSTALL POSTGRESSQL and CREATE A DATABASE WITH A USER having permissions to create databases.
Follow instructions [here](http://wiki.sylfen.com/sql/)

In [ ]:
conf.DB_PARAMETERS={
    'host'     : "localhost",
    'port'     : "5432",
    'dbname'   : "db_test",
    'user'     : "test_user",
    'password' : "test"
}

conf.DB_TABLE  = 'realtimedata_test'

The next step enables to create the adequate table if it does not exist in your postgressql database.

In [ ]:
from sylfenUtils.utils import DataBase
DataBase().create_sql_table(conf.DB_PARAMETERS,conf.DB_TABLE)

Precise where the data will be stored on your machine, which timezone, how big the database will be and the logs folder.

In [ ]:
conf.TZ_RECORD = 'CET'## the timezone used to park the data
conf.PARKING_TIME  = 60*10 #### how often (in seconds) the data should be parked from the buffer database(the database is then flushed)
BASE_FOLDER = os.getenv('HOME')+'/dummy_project/'
if not os.path.exists(BASE_FOLDER):os.mkdir(BASE_FOLDER)#create the folder it if does not exist
conf.FOLDERPKL   = BASE_FOLDER + '/data_daily/' ### where the (daily)parked data will be stored
if not os.path.exists(conf.FOLDERPKL):os.mkdir(conf.FOLDERPKL)#create the folder it if does not exist
conf.LOG_FOLDER  = BASE_FOLDER + '/log/'
if not os.path.exists(conf.LOG_FOLDER):os.mkdir(conf.LOG_FOLDER)#create the folder it if does not exist

Your conf in now ready and can be seen and used independenly. As long as your are not happy with your conf work on it before
going to the next step.

In [ ]:
conf.dummy_df_plc

# 2. DUMP DATA 
You learn here how to istanciate your dumper to record data as daily parked format. 

## 2.1 Create modbus devices, add them to your DUMPER which will record data as parked format
Your first device is the dummy device whose ip adress, byte order, word order are known. You just need to enter 
- ip of the device
- running port
- the name of the device(you can choose any, this has no effect)
- the plc dataframe *dfplc*
- the modbus map 
- the words(*wo*) and bytes(*bo*) endianess. Only 'little' or 'big' can be used. 
- the frequence *freq* of acquisition in seconds. How often you want to fetch the data.

In [ ]:
from sylfenUtils.comUtils import ModbusDevice
dummy_device=ModbusDevice(ip='localhost',port=3580,device_name='dummy_device',
    dfplc=conf.dummy_df_plc,modbus_map=conf.dummy_modbus_map,bo='big',wo='big',freq=2)

## 2.2 Start the SIMULATOR of the modbus device (optionnal)
As we don't have the dummy device delivering data with the modbus protocole, we will create a simulator of the device based on the same modbus map and we will run the modbus server to serve data (random data). 
For your real project you can of course skip this step if the device is available on your network.

In [ ]:
from sylfenUtils.Simulators import SimulatorModeBus
dummy_simulator=SimulatorModeBus(port=dummy_device.port,modbus_map=dummy_device.modbus_map,bo=dummy_device.byte_order,wo=dummy_device.word_order)
dummy_simulator.start()

## 2.3 make sure you can connect to the device and COLLECT ALL THE DATA from your modbus device in real time. (not necessary in operation)

If you don't know the endianness you can instanciate your device anyway and yse the function **quick_modbus_single_register_decoder** with one register to guess the endianess.

In [ ]:
dummy_device.connectDevice()
dummy_device.quick_modbus_single_register_decoder(10,2,'float32',unit=1);

here we see that the only reasonable value is for the combination byte_order='big' and words_order='little because all the others have extrem values. If this is still not unambiguous after one call. You can call it a second time. 

In [ ]:
dummy_device.quick_modbus_single_register_decoder(10,2,'float32',unit=1);

now we can see that all the others combinations have their values changed too radically. So it is very very likely that the correct encoding is 'big', 'big'.

In [ ]:
tags=dummy_device.dfplc.index.to_list()
dummy_device.connectDevice()
data=dummy_device.collectData('CET',tags)## do not forget to precise the time zome
data

## 2.4 Check that you can INSERT DATA INTO THE DATABASE (not necessary in operation)
The database serves as a buffer for the realtime acquisition while simultaneously having the possibility to access the data. 
Check that the data were correclty inserted by retrieving all the data of your database.

In [ ]:
dummy_device.insert_intodb(conf.DB_PARAMETERS,conf.DB_TABLE,'CET',tags)

Check that your database was filled with the data

In [ ]:
def quick_check_db():
    import psycopg2
    connReq = ''.join([k + "=" + v + " " for k,v in conf.DB_PARAMETERS.items()])
    dbconn = psycopg2.connect(connReq)
    sqlQ ="select * from " + conf.DB_TABLE +" order by timestampz asc;"
    df = pd.read_sql_query(sqlQ,dbconn,parse_dates=['timestampz'])
    print(df)
quick_check_db()

## 2.5 Instanciate your DUMPER
With your devices you are ready to start your dumper. You need to precise : 
- a dictionnary of the devices 
- the folder where the data will be stored.
- the parameters of the database 
- the time window of the database in seconds.
- the name of the table in the database *dbTable*
- the timezone used to record the data *tz_record*(by default 'CET')
- the filename of the log file(by default None)

In [ ]:
from sylfenUtils.comUtils import SuperDumper_daily
DEVICES = {
    'dummy_device':dummy_device,
}
log_file_name=conf.LOG_FOLDER+'/dumper.log' ## give a name to your logger. 
log_file_name=None ## if you want either to have the information in the console use None.
dumper=SuperDumper_daily(DEVICES,conf.FOLDERPKL,conf.DB_PARAMETERS,conf.PARKING_TIME,
    dbTable=conf.DB_TABLE,tz_record=conf.TZ_RECORD,log_file=log_file_name)

## 2.6 Park the database first before starting to dump data in case the database is already big. 

In [ ]:
dumper.park_database()

## 2.7 start to dump data.

In [ ]:
dumper.start_dumping()

check that it is correctly feeding the database

In [ ]:
quick_check_db()

Remarks:
- opcua device are also available
- if you want to create a new device class that works neither with modbus nor with OPCUA protocole you can create a children class of comUtils.Device. Just make sure : 
    - to rewrite a function **collectData** that collect all the data from the plc dataframe of the device.
    - a function **connectDevice** that connects to the device.


# 3. READ THE DATA in REAL TIME
Now you can load your parked data to process and visualize them.

## 3.1 Instanciate the VISUALISER object
you need to enter almost all the same parameters as for the dumper. You may wonder and find it unconvenient because we could have included this object in the dumper which in that case you won't have to recreate an other object. The reasons for not having proceeded this way are :
- dissociation of the objects so that you can visualise data from a folder even if the dumper is not running or not working. Loading historical data is possible without the dumper. 
- it simplifies the architecture of the code. Modularity brings complexity but improve order, robustness, maintainability and readability.   

In [ ]:
from sylfenUtils.comUtils import VisualisationMaster_daily
cfg=VisualisationMaster_daily(
    conf.FOLDERPKL,
    conf.DB_PARAMETERS,
    conf.PARKING_TIME,
    dbTable=conf.DB_TABLE,
    tz_record=conf.TZ_RECORD
)

There is now a SMALL SHORTCOMING HERE OF THE LIBRARY. The visualiser needs the whole plc dataframe with all the tags of all your devices. 
You should not need to use the dumper to instanciate the visualiser. 
It would be better to concatenate the plc_dataframe of all your devices in one plc_dataframe.

In [ ]:
cfg.dfplc=dumper.dfplc ### required to use the function getTagsTU
cfg.listUnits=list(cfg.dfplc['UNITE'].unique())

## 3.2 Grab you tags 
if your plc dataframe is big you may need not remember all the tags ny heart and you would like to grab the tags with key expressions. 
Use the fonction **getTagsTU** of the object to look for tags using a regular expressions. 
For example we want to get all the pressure and temperature sensors that works with water(H2O) in our system.

In [ ]:
tags=cfg.getTagsTU('[PT]T.*H2O')
tags

## 3.3 LOAD the data precising the resampling method and resampling time. Here we want the last 2 hours
Of course you can get the data from anytime to anytime given t0,t1 as timestamps.

In [ ]:
t1=pd.Timestamp.now(tz='CET')
t0=t1-pd.Timedelta(hours=2)
df=cfg.loadtags_period(t0,t1,tags,rs='2s',rsMethod='mean')
df

## 3.4 PLOT the data with a standard multi unit scale graph
You can of course also plot the data with your own function. The advantages of this function is that it can plot the data using different y-scales.

In [ ]:
from sylfenUtils.utils import Graphics
Graphics().multiUnitGraph(df).show()

# 4. DEPLOY THE WEB INTERFACE
The interface enables other people(or you) to access the data in a convenient way from the web plateform.

## 4.1 PREPARE the folder of your app
For the application to work (with flask) it is needed to have a folder *templates/* and *static/* in the *root_folder*.

Chose your root folder 

In [ ]:
BASE_FOLDER = os.getenv('HOME')+'/dummy_project/'
root_folder=os.path.dirname(BASE_FOLDER)+'/dummy_app/'
root_folder
if not os.path.exists(root_folder):os.mkdir(root_folder)

Make a symbolic link of the folder *templates* of the sylfenUtils library into your root folder.

In [ ]:
import subprocess as sp
import sylfenUtils
sylfenUtils_env_dir=os.path.dirname(sylfenUtils.__file__)
templates_dir=sylfenUtils_env_dir + '/templates'
templates_folder=root_folder + '/templates'
if os.path.exists(templates_folder):os.remove(templates_folder)
sp.run('ln -s '+templates_dir + ' ' + root_folder,shell=True)

Make a symbolic link of the folder *static/lib* into your root folder.

In [ ]:
static_folder=root_folder+'/static/'
if not os.path.exists(static_folder):os.mkdir(static_folder) #create folder if it does not exist
lib_dir=sylfenUtils_env_dir + '/static/lib'
lib_folder=static_folder + '/lib'
if os.path.exists(lib_folder):os.remove(lib_folder)
sp.run('ln -s ' + lib_dir + ' ' + static_folder,shell=True)

## 4.2 Configure the app with some default settings.

In [ ]:
init_parameters={
    'tags':cfg.getTagsTU('[PTF]T.*H2O'),
    'fig_name':'temperatures, pressures, and mass flows',
    'rs':'30s',
    'time_window':str(2*60),
    'delay_minutes':0,
    'log_versions':None #you can enter the relative path of (in folder static) a .md file summarizing some evolution in your code.
}

## 4.3 Instanciate the DASHBOARD

In [ ]:
from sylfenUtils import dashboard
APP_NAME='dummy_app'
dash=dashboard.Dashboard(
    cfg,
    conf.LOG_FOLDER,
    root_folder,
    app_name=APP_NAME,
    init_parameters=init_parameters,
    plot_function=cfg.utils.multiUnitGraph, ## you can use your own function to display the data 
    version_dashboard='1.0')

dash.helpmelink='' ### you can precise a url link on how to use the web interface
dash.fig_wh=780### size of the figure

If you need to extend some functionnality you can do it here.
For example you can ask the back end to print a text with front-end get '/example'

In [ ]:
@dash.app.route('/example', methods=['GET'])
def example_extension():
    print('this is a test')

## 4.4 Start the app

In [ ]:
port_app=30000
dash.app.run(host='0.0.0.0',port=port_app,debug=False,use_reloader=False)
url_app='localhost:'+str(port_app)+'/'+APP_NAME # debug and use_reloader as True can only be used in developpment work but not in production.
print('your app is now available at :\n',url_app)

- For for the app to be seen by others on your private network the url is : *your_private_ip_adress*>:*your_port_app*/*your_app_name*
- to deploy it to the world, open the port of the app on your routeur and the url is : *your_public_ip_adress*>:*your_port_app*/*your_app_name*

However the library Flask which is serving the website is not very stable. It is better to use wsgi and gunicorn to run the app and start the service with systemctl on a linux server. Then use ngnix(or apache) to make the redirection via a myapp.conf file in the sites_enabled of the nginx folder. You can use your domain name or subdomain name directly them so that the url becomes :
www.*your_domain_name*/*your_app_name*